In [1]:
#Import Libraries

import os
from itertools import chain
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.io.json import json_normalize
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
#pip install spacy==2.2.3
#python -m spacy download en_core_web_sm
#pip install beautifulsoup4==4.9.1
#pip install textblob==0.15.3
from dataclasses import dataclass
import spacy.cli
import spacy
#spacy.cli.download("en_core_web_sm",'--no-deps')

#download("en_core_web_sm")


from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sheet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = r"C:\Users\sheet\Dataset\Tweets\tweets_ws22_v1.json"

with open(path) as f:
   
    dump = json.loads(f.read())
    org_df = pd.json_normalize(dump)
    org_df
    
final_df = pd.DataFrame() 

for i in range(0,len(dump)):
    temp_topic_df = pd.json_normalize(dump[i]['labels']['topic'])
    temp_sentiment_df = pd.json_normalize(dump[i]['labels']['sentiment'])
    temp_topic_df.rename(columns={'id':'topic_id','tweet_id':'topic_tweet_id','user_id':'topic_user_id','ground_truth':'topic_ground_truth'}, inplace = True)
    temp_sentiment_df.rename(columns={'id':'sentiment_id','tweet_id':'sentiment_tweet_id','user_id':'sentiment_user_id','ground_truth':'sentiment_ground_truth'}, inplace = True)
    df_join_topic_sent = temp_topic_df.join(temp_sentiment_df)
    final_df = final_df.append(df_join_topic_sent)
    #final_df = pd.concat([org_df,Joined_df],axis = 1)
    
    
cols = final_df.columns
org_df[cols] = final_df[cols].values


data =  org_df.drop(columns=['labels.topic','labels.sentiment','topic_tweet_id','sentiment_tweet_id','author_id','source','topic_user_id','topic_ground_truth','language','id','source_id','relevant','sentiment_ground_truth','sentiment_id','topic_id','sentiment_user_id'])
data


,source_created_at,text,longitude,latitude,topic,sentiment
0,2020-09-18 21:56:20.798000,@DSisourath The Thameslink core between London...,-0.12574,51.50853,service,negative
1,2020-10-13 07:31:53.122000,@DulwichHistory Loving the complaint about peo...,-0.12574,51.50853,delays,negative
2,2020-10-26 19:27:24.695000,@SW_Help .And yet you have no toilets on some ...,-0.12574,51.50853,toilets,negative
3,2020-10-26 19:28:49.281000,@SW_Help you have no toilets on some of your t...,-0.12574,51.50853,toilets,negative
4,2020-09-28 11:59:41.212000,@SpeedySticks007 @MrNeilJH @TLRailUK @christia...,-1.09125,50.79899,seats,neutral
...,...,...,...,...,...,...
16944,2019-07-11 07:34:35,Haha oh man the audio corruption on @TLRailUK ...,NaN,NaN,announcements,neutral
16945,2020-08-10 11:19:10.181000,@TLRailUK SweetIs there a plug to charge my ph...,-0.12574,51.50853,plugs,neutral
16946,2020-08-29 09:51:10.833000,@TLRailUK now there are far fewer commuters ha...,-0.12574,51.50853,tables,neutral
16947,2020-11-02 12:06:06.967000,@geofftech I am voting for Thameslink. 1. in a...,-0.12574,51.50853,toilets,neutral


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16949 entries, 0 to 16948
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source_created_at  16949 non-null  object 
 1   text               16949 non-null  object 
 2   longitude          1425 non-null   float64
 3   latitude           1425 non-null   float64
 4   topic              16949 non-null  object 
 5   sentiment          16949 non-null  object 
dtypes: float64(2), object(4)
memory usage: 794.6+ KB


In [4]:
#pip install text-hammer==0.1.4

In [5]:
import text_hammer as th

In [6]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
#     df[column] = df[column].progress_apply(lambda x: ps.remove_stopwords(x))

    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))

    return(df)

In [7]:
cleaned_df = text_preprocessing(data,'text')
cleaned_df.head()

  0%|          | 0/16949 [00:00<?, ?it/s]

  0%|          | 0/16949 [00:00<?, ?it/s]

  0%|          | 0/16949 [00:00<?, ?it/s]

C:\Users\sheet\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


  0%|          | 0/16949 [00:00<?, ?it/s]

  0%|          | 0/16949 [00:00<?, ?it/s]

,source_created_at,text,longitude,latitude,topic,sentiment
0,2020-09-18 21:56:20.798000,dsisourath the thameslink core between london ...,-0.12574,51.50853,service,negative
1,2020-10-13 07:31:53.122000,dulwichhistory loving the complaint about peop...,-0.12574,51.50853,delays,negative
2,2020-10-26 19:27:24.695000,sw_help and yet you have no toilets on some of...,-0.12574,51.50853,toilets,negative
3,2020-10-26 19:28:49.281000,sw_help you have no toilets on some of your tr...,-0.12574,51.50853,toilets,negative
4,2020-09-28 11:59:41.212000,speedysticks007 mrneiljh tlrailuk christianwol...,-1.09125,50.79899,seats,neutral


In [8]:
#Preparing data for Bert Architecture format

In [9]:
cleaned_df['num_words'] = cleaned_df.text.apply(lambda x:len(x.split()))
cleaned_df.num_words.max()

87

In [10]:
# changing the data type to the category to encode into codes 
cleaned_df['sentiment'] = cleaned_df.sentiment.astype('category')
cleaned_df.sentiment

0        negative
1        negative
2        negative
3        negative
4         neutral
           ...   
16944     neutral
16945     neutral
16946     neutral
16947     neutral
16948     neutral
Name: sentiment, Length: 16949, dtype: category
Categories (3, object): ['negative', 'neutral', 'positive']

In [11]:
cleaned_df['sentiment']  =  cleaned_df.sentiment.cat.codes
cleaned_df.sentiment

0        0
1        0
2        0
3        0
4        1
        ..
16944    1
16945    1
16946    1
16947    1
16948    1
Name: sentiment, Length: 16949, dtype: int8

In [12]:
cleaned_df.head()

,source_created_at,text,longitude,latitude,topic,sentiment,num_words
0,2020-09-18 21:56:20.798000,dsisourath the thameslink core between london ...,-0.12574,51.50853,service,0,21
1,2020-10-13 07:31:53.122000,dulwichhistory loving the complaint about peop...,-0.12574,51.50853,delays,0,19
2,2020-10-26 19:27:24.695000,sw_help and yet you have no toilets on some of...,-0.12574,51.50853,toilets,0,27
3,2020-10-26 19:28:49.281000,sw_help you have no toilets on some of your tr...,-0.12574,51.50853,toilets,0,29
4,2020-09-28 11:59:41.212000,speedysticks007 mrneiljh tlrailuk christianwol...,-1.09125,50.79899,seats,1,37


In [13]:
from tensorflow.keras.utils import to_categorical
to_categorical(cleaned_df.sentiment,3)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [14]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(cleaned_df, test_size = 0.3, random_state = 42, stratify = cleaned_df.sentiment)

In [15]:
data_train.shape


(11864, 7)

In [16]:
data_test.shape

(5085, 7)

In [17]:
data_train['sentiment'].value_counts()

0    7440
1    4255
2     169
Name: sentiment, dtype: int64

In [18]:
#!pip install transformers
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [40]:
# Tokenize the input  

x_train = tokenizer(
    text=data_train.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [41]:
from sklearn.utils import class_weight
#Calculate weights 
weights = class_weight.compute_class_weight('balanced',np.unique(data_train['sentiment']),data_train['sentiment'])
weights

#transform array to dictionary
weights = dict(enumerate(weights))

C:\Users\sheet\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2], y=11829    0
611      0
10549    0
12659    1
6450     0
        ..
685      0
15314    0
4881     0
5541     0
14699    0
Name: sentiment, Length: 11864, dtype: int8 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


cleaned_df['sentiment']  =  cleaned_df.sentiment.cat.codes
cleaned_df.sentiment

# Model

In [43]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [44]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [45]:
max_len = 70
   
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    


embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(3,activation = 'sigmoid')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

#return model, tokenizer


In [46]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 70)]         0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 70,                                          

In [47]:
tf.keras.utils.plot_model(model,show_shapes = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [48]:
optimizer = Adam(learning_rate = 5e-05, epsilon=1e-08, decay =0.01,clipnorm= 1.0)

loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

#compile model
model.compile(optimizer =optimizer,loss =loss,metrics =metric)

In [49]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

In [50]:
#Model fit and evaluation

In [ ]:
train = model.fit(x= {'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']},
                  y=to_categorical(data_train.sentiment,3),
                  validation_data= ({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']},to_categorical(data_test.sentiment,3)),
                  epochs =1,
                  batch_size =70
                 )

C:\Users\sheet\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
C:\Users\sheet\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


 36/170 [=====>........................] - ETA: 4:09:09 - loss: 0.7304 - balanced_accuracy: 0.6246

In [ ]:
#Model prediction without balancing

In [ ]:
predict = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [ ]:
y_predicted = np.argmax(predict,axis =1)

In [ ]:
data_test.sentiment #ytrue

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(data_test.sentiment,y_predicted ))

In [ ]:
# testing the model

In [ ]:
texts = input(str('input the text'))

x_val = tokenizer(
        text=input_text,
        add_special_tokens=True,
        max_length=70,truncation=True,
        padding='max_length', 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,verbose = True)
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100 
validation

In [ ]:
encoded_dict  = {'negative':0,'neutral':1, 'positive':2}

In [ ]:
for key,value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

In [ ]:
#  after balancing using class weight

In [ ]:
train = model.fit (x= {'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']},
                  y=to_categorical(data_train.sentiment,3),
                  validation_data= ({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']},to_categorical(data_test.sentiment,3)),
                  epochs =1,
                  batch_size =36,
                  class_weight =weights
                 )

In [ ]:
predicted = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [ ]:
y_predicted = np.argmax(predicted,axis =1)